In [7]:
from typing import Annotated
from dotenv import load_dotenv
load_dotenv()

from typing_extensions import TypedDict
from langchain.chat_models import init_chat_model
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

In [8]:
from langchain.chat_models import init_chat_model

# 🎯 Purpose

This code defines a simple **chatbot workflow** using LangGraph.  
- It initializes the Gemini 2.0 Flash LLM.  
- Defines a conversation `State` to hold messages.  
- Creates a chatbot node that generates responses using the LLM.  
- Builds a state graph with a linear flow (`START → chatbot_node → END`).  
- Compiles the graph into an executable workflow that can maintain conversation history and generate replies.  


In [9]:
llm = init_chat_model("google_genai:gemini-2.0-flash")

class State(TypedDict):
    messages: Annotated[list, add_messages]

def chatbot(state: State) -> State:
    return {"messages": [llm.invoke(state["messages"])]}

builder = StateGraph(State)
builder.add_node("chatbot_node", chatbot)

builder.add_edge(START, "chatbot_node")
builder.add_edge("chatbot_node", END)

graph = builder.compile()



## This code demonstrates a **one-shot execution** of the chatbot graph.

**Characteristics:**
- Executes the graph once for a single user query.  
- Shows both input and output for that query.  
- Demonstrates how to send a message into the chatbot graph and inspect the response.  
- State (conversation history) exists but is **not reused** after this call.  


In [12]:
message = {"role": "user", "content": "Who walked on the moon for the first time? Print only the name"}
#message = {"role": "user", "content": "What is the latest price of MSFT stock?"}
response = graph.invoke({"messages":[message]})

response["messages"]

[HumanMessage(content='Who walked on the moon for the first time? Print only the name', additional_kwargs={}, response_metadata={}, id='03fa9fb7-7b6d-4639-a7a8-ee5dc8b53d30'),
 AIMessage(content='Neil Armstrong', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--20c75bb1-1e93-4b88-8e35-877f710cce57-0', usage_metadata={'input_tokens': 14, 'output_tokens': 3, 'total_tokens': 17, 'input_token_details': {'cache_read': 0}})]


## This code creates an **interactive chatbot loop** using the compiled graph.

**Characteristics:**
- Allows continuous multi-turn conversation with the user.  
- Maintains and updates the conversation **state** across turns (chatbot remembers context).  
- User input is taken dynamically until `"quit"` or `"exit"` is entered.  
- The bot responds with the latest message from the conversation history.  
- Demonstrates a **realistic chatbot behavior** rather than a single one-shot query.  


In [13]:
state = None
while True:
    in_message = input("You: ")
    if in_message.lower() in {"quit","exit"}:
        break
    if state is None:
        state: State = {
            "messages": [{"role": "user", "content": in_message}]
        }
    else:
        state["messages"].append({"role": "user", "content": in_message})

    state = graph.invoke(state)
    print("Bot:", state["messages"][-1].content)

Bot: Neil Armstrong
Bot: I am a large language model, trained by Google.
Bot: I don't have a name in the way a person does. You can think of me as a tool.
Bot: Your first question was: "Who walked on the moon for the first time?"
Bot: Okay, goodbye! Let me know if you have any other questions later.
